In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torchtext.vocab as vocab
from torchtext.vocab import GloVe
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import time
import torch.optim as optim
import matplotlib.pyplot as plt
from ordered_set import OrderedSet

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
special_tags=["<sos>","<eos>","<unk>","<pad>"]
symbols = ["(", ")", ",","|","const_"]
symbols.extend(range(10))
pos_special_tokens_prob=[0,0,0,0]
pos_special_tokens_sol=[0,0,0,0]
embed_dim=100
beam_size=10

In [ ]:
def collate(batch):
    
    max_len_problem = max([len(sample[0]) for sample in batch])
    max_len_solution = max([len(sample[1]) for sample in batch])
    
    padded_prob = torch.empty((len(batch), max_len_problem), dtype=torch.long)
    padded_prob.fill_(pos_special_tokens_prob[3])
    padded_sol = torch.empty((len(batch), max_len_solution), dtype=torch.long)
    padded_sol.fill_(pos_special_tokens_sol[3])
    ans=torch.zeros(len(batch))

    for idx in range(len(batch)):
        
        ans[idx]=batch[idx][2]
        padded_prob[idx, :len(batch[idx][0])] = torch.LongTensor(batch[idx][0])
        padded_sol[idx, :len(batch[idx][1])] = torch.LongTensor(batch[idx][1])
        
    return (padded_prob,padded_sol,ans)


In [ ]:
class load_data_train(Dataset):
    def __init__(self,json_path):
        self.path=json_path
        self.data=[]
        self.loaddata()
        self.problem_unique_words,self.sol_unique_words=self.gen_all_unique_words()
        self.problem_word2int = {word: i for i, word in enumerate(self.problem_unique_words)}
        self.problem_int2word = {i: word for word, i in self.problem_word2int.items()}
        self.sol_word2int = {word: i for i, word in enumerate(self.sol_unique_words)}
        self.sol_int2word = {i: word for word, i in self.sol_word2int.items()}
        self.max_problem_len=self.get_max_len()
        self.get_special_pos_prob()
        self.get_special_pos_sol()
    def get_special_pos_prob(self):
        for i,t in enumerate(special_tags):
            pos_special_tokens_prob[i]=self.problem_word2int[t]
            
    def get_special_pos_sol(self):
        for i,t in enumerate(special_tags):
            pos_special_tokens_sol[i]=self.sol_word2int[t]
            
    def gen_all_unique_words(self):
        u1=OrderedSet(special_tags)
        u2=OrderedSet(symbols+special_tags)
        for i,(prob,sol,a) in enumerate(self.data):
            for word in prob.split():
                u1.add(word)
            operations = sol.split("|")
            l=self.tokanize_sol(i)
            for t in l:
                u2.add(t)
        return u1,u2
    
    def tokanize_problem(self,i):
        return self.data[i][0].split()
    
    def tokanize_sol(self,i):
        l=[]
        operations = self.data[i][1].split("|")
        for j,operation in enumerate(operations):
            if not operation:  
                continue
            operation_name = operation.split("(")[0]
            l.append(operation_name) 
            l.append("(")
            content = operation[operation.find("(")+1:operation.find(")")]
            tokens = content.split(",")
            new=[]
            for i, token in enumerate(tokens):
                new.append(token)
                if i < len(tokens) - 1:
                    new.append(",")
            l.extend(new)
            l.append(")")
            if j < len(operations) - 1:
                    l.append("|")
        return l
    
    def get_max_len(self):
        m=0
        for (p,s,a) in self.data:
            for p1 in p:
                m=max(m,len(p1.split()))
        return m

    def loaddata(self):
        with open(self.path, 'r') as f:
            data = json.load(f)
            for i in data:
                #list(i["Problem"].split())
#                 p=str(i["Problem"]).split()
                self.data.append((i["Problem"],i["linear_formula"],i["answer"]))
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        problem=["<sos>"]+self.tokanize_problem(i)+["<eos>"]
        sol=["<sos>"]+self.tokanize_sol(i)+["<eos>"]
        problem = [self.problem_word2int[q] if q in self.problem_word2int else self.problem_word2int["<unk>"] for q in problem]
        sol = [self.sol_word2int[q] if q in self.sol_word2int else self.sol_word2int["<unk>"] for q in sol]
        return ((problem,sol,self.data[i][2]))



In [ ]:
class load_data_test(Dataset):
    def __init__(self,json_path,train):
        self.path=json_path
        self.data=[]
        self.loaddata()
        self.problem_unique_words,self.sol_unique_words=train.problem_unique_words,train.sol_unique_words
        self.problem_word2int = train.problem_word2int
        self.problem_int2word = train.problem_int2word
        self.sol_word2int = train.sol_word2int
        self.sol_int2word =train.sol_int2word
    
    def tokanize_problem(self,i):
        return self.data[i][0].split()
    
    def tokanize_sol(self,i):
        l=[]
        operations = self.data[i][1].split("|")
        for j,operation in enumerate(operations):
            if not operation:  
                continue
            operation_name = operation.split("(")[0]
            l.append(operation_name) 
            l.append("(")
            content = operation[operation.find("(")+1:operation.find(")")]
            tokens = content.split(",")
            new=[]
            for i, token in enumerate(tokens):
                new.append(token)
                if i < len(tokens) - 1:
                    new.append(",")
            l.extend(new)
            l.append(")")
            if j < len(operations) - 1:
                    l.append("|")
        return l

    def loaddata(self):
        with open(self.path, 'r') as f:
            data = json.load(f)
            for i in data:
                #list(i["Problem"].split())
#                 p=str(i["Problem"]).split()
                self.data.append((i["Problem"],i["linear_formula"],i["answer"]))
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        problem=["<sos>"]+self.tokanize_problem(i)+["<eos>"]
        sol=["<sos>"]+self.tokanize_sol(i)+["<eos>"]
        problem = [self.problem_word2int[q] if q in self.problem_word2int else self.problem_word2int["<unk>"] for q in problem]
        sol = [self.sol_word2int[q] if q in self.sol_word2int else self.sol_word2int["<unk>"] for q in sol]
        return ((problem,sol,self.data[i][2]))



In [ ]:
train_path="/kaggle/input/new-wp/data/train.json"
train_data=load_data_train(train_path)
train_loader = DataLoader(train_data, batch_size=64, shuffle=False, collate_fn=collate)

test_path="/kaggle/input/new-wp/data/test.json"
test_data=load_data_test(test_path,train_data)
test_loader = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=collate)



validation_path="/kaggle/input/new-wp/data/dev.json"
val_data=load_data_test(validation_path,train_data)
validation_loader = DataLoader(val_data, batch_size=64, shuffle=False, collate_fn=collate)
validation_loader1 = DataLoader(val_data, batch_size=1, shuffle=False, collate_fn=collate)


In [ ]:
class GloveEmbeddings():
    def __init__(self, embed_dim, word2idx):
        self.embed_dim = embed_dim
        self.word2idx = word2idx
        self.vocab_size = len(word2idx)
    
    def get_embedding_matrix(self):
        glove = GloVe(name='6B', dim=self.embed_dim)
        embedding_matrix = torch.zeros((self.vocab_size, self.embed_dim)) #init zeros
        
        for i in pos_special_tokens_prob[:3]:
            embedding_matrix[i] = torch.randn(self.embed_dim)
            
        for k, v in self.word2idx.items():        
            if k in glove.stoi:
                embedding_matrix[v] = torch.tensor(glove.vectors[glove.stoi[k]])
            elif k not in special_tags:
                embedding_matrix[v] = embedding_matrix[pos_special_tokens_prob[2]]
#         embedding_matrix[]
        return embedding_matrix

In [ ]:
glove=GloveEmbeddings(200,train_data.problem_word2int)

In [ ]:
embedding=glove.get_embedding_matrix()

In [ ]:

class AttentionNetwork(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionNetwork, self).__init__()
        self.attention_fc = nn.Linear(hidden_dim * 3, hidden_dim)
        self.vector_projection = nn.Linear(hidden_dim, 1, bias=False)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, last_hidden_state, encoder_outputs):
        # Correctly repeating the hidden state to match the batch size of the encoder outputs
        repeated_hidden = last_hidden_state.repeat(encoder_outputs.size(1), 1, 1).transpose(0, 1)
        energy = torch.cat((repeated_hidden, encoder_outputs), dim=2)
        energy = torch.tanh(self.attention_fc(energy))
        attention_scores = self.vector_projection(energy).squeeze(2)
        attention_weights = self.softmax(attention_scores)
        context_vector = attention_weights.unsqueeze(1).bmm(encoder_outputs)
        return context_vector, attention_weights

In [ ]:

    
          
class LSTMEncoder(nn.Module):
    def __init__(self, embedding_dim, hidden_units=256, embed_matrix=None):
        super(LSTMEncoder, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_units = hidden_units
        self.embedding = nn.Embedding.from_pretrained(embed_matrix, padding_idx=pos_special_tokens_prob[3])
        self.dropout = nn.Dropout(0.5) 
        self.lstm = nn.LSTM(embedding_dim, hidden_units, num_layers=1, batch_first=True, 
                            dropout=0.5, bidirectional=True)
        self.hidden_layer = nn.Linear(hidden_units * 2, hidden_units)  
        self.cell_layer = nn.Linear(hidden_units * 2, hidden_units)

    def forward(self, inputs):
        embedded_inputs = self.dropout(self.embedding(inputs))  
        lstm_out, (hidden, cell) = self.lstm(embedded_inputs)
        hidden = self.hidden_layer(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
        cell = self.cell_layer(torch.cat((cell[0:1], cell[1:2]), dim=2))
        return lstm_out, (hidden, cell)




In [ ]:
 
    
class LSTMAttnDecoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, units=256):
        super(LSTMAttnDecoder, self).__init__()
        self.embed_layer = nn.Embedding(vocab_size, embedding_dim, padding_idx=pos_special_tokens_sol[3])  
        self.dropout_layer = nn.Dropout(0.5)
        self.attn_layer = AttentionNetwork(units)
        self.lstm = nn.LSTM(embedding_dim + units * 2, units, num_layers = 1, bidirectional = False, dropout=0.5, batch_first=True)
        self.out = nn.Linear(units, vocab_size)

    def forward(self, input_seq, hidden_state, encoder_outputs):
        embedded = self.dropout_layer(self.embed_layer(input_seq)).unsqueeze(1)
        attn_context, attn_weights = self.attn_layer(hidden_state[0], encoder_outputs)
        lstm_input = torch.cat([embedded, attn_context], dim=2)
        lstm_out, (h_n, c_n) = self.lstm(lstm_input, hidden_state)
        output = self.out(lstm_out)
        return output, (h_n, c_n)


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, embedding=embedding,embedding_dim=200,max_len_pred=None):
        super(Seq2Seq, self).__init__()
        self.lstm_encoder=LSTMEncoder(embedding_dim=embedding_dim,embed_matrix=embedding)
        self.lstm_decoder=LSTMAttnDecoder(vocab_size=max_len_pred,embedding_dim=embedding_dim)
        self.decoder_vocab_size=max_len_pred
    def forward(self, source_seq, target_seq, tf=0.9):
        src_batch_size = source_seq.size(0)
        tgt_seq_len = target_seq.size(1)
        
        encoder_states, (encoder_hidden, encoder_cell) = self.lstm_encoder(source_seq)

        decoder_vocab_size = self.decoder_vocab_size
        decoder_outputs = torch.zeros(src_batch_size, tgt_seq_len, decoder_vocab_size)
        predicted_sequence = torch.zeros(src_batch_size, tgt_seq_len)

        decoder_input_token = target_seq[:, 0]  # Initial decoder input
        predicted_sequence[:, 0] = decoder_input_token
        
        for step in range(1, tgt_seq_len):
            decoder_output, (encoder_hidden, encoder_cell) = self.lstm_decoder(decoder_input_token, (encoder_hidden, encoder_cell),encoder_states)
            decoder_output = decoder_output.squeeze(1)
            decoder_outputs[:, step, :] = decoder_output
            use_teacher_forcing = np.random.random() < tf
            decoder_input_token = target_seq[:, step] if use_teacher_forcing else decoder_output.argmax(dim=1)
            predicted_sequence[:, step] = decoder_output.argmax(dim=1)

        return decoder_outputs, predicted_sequence


In [ ]:
model=Seq2Seq(embedding=embedding,embedding_dim=200,max_len_pred=len(train_data.sol_unique_words)).to(device)
def train(epochs):
    st=time.time()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    train_l=[]
    val_l=[]
    for epoch in range(epochs):
        print(f"=======================epoch {epoch}================================")
        model.train()
        l=[]
        for j,data1 in enumerate(train_loader):
            optimizer.zero_grad()
            x=data1[0].to(device)
            y=data1[1].to(device)
            o,w=model(x,y,tf=0.9)
            o = o.reshape(-1, o.shape[2]).to(device)
            y_orig = y.reshape(-1).to(device)
            loss = criterion(o, y_orig)
            loss.backward()
            optimizer.step()
            l.append(loss.item())
        train_l.append(np.mean(l))
        model.eval()
        l=[]
        for j,data1 in enumerate(validation_loader):
            x=data1[0].to(device)
            y=data1[1].to(device)
            o,w=model(x,y,tf=0)
            o = o.reshape(-1, o.shape[2]).to(device)
            y_orig = y.reshape(-1).to(device)
            loss = criterion(o, y_orig)
            l.append(loss.item())
        val_l.append(np.mean(l))
        print(f"epoch : {epoch} train_loss : {train_l[-1]} val_loss: {val_l[-1]} time taken : {(time.time()-st)/60}  ")
        print("model saved!!")
        torch.save(model, 'modelB_inloop.pth')
    return train_l,val_l

In [ ]:
# import gc
# gc.collect()

# # Clear memory allocated by PyTorch
# torch.cuda.empty_cache()

In [ ]:
train_l,val_l=train(50)
print("train completed!!")

In [ ]:
torch.save(model, 'modelB_final_50_epoch.pth')

In [ ]:
def generate_csv(model,data,loader):
    model.eval()
    prediction=[]
    for batch in loader:
        x=batch[0].to(device)
        y=batch[1].to(device)
        o,w=model(x,y,tf=0)
        prediction.append(w)
    predicted_data_strings=[]
    for batch in prediction:
        for one_data in batch:
            one_data=one_data.to(torch.int)
            one_data_in_string=""
            conv_int_word=[]
            for pos,value in enumerate(one_data):
                conv_int_word.append(data.sol_int2word[value.item()])
            predicted_data_strings.append(conv_int_word)
    return predicted_data_strings

In [ ]:
def extract_sol(data2str):
    complete_data=[]
    for each_data in data2str:
        s=""
        flag=0
        for item in each_data[1:]:
            if(item=="<eos>" or item=="<pad>"):
                complete_data.append(s)
                flag=1
                break
            else:
                s+=str(item)
        if(flag==0):
            complete_data.append(s)
    return complete_data

In [ ]:
def generate_sol(int2data,name,data_name):
    json_data=[]
    for k,s in zip(data_name.data,int2data):
        d={
            "Problem":k[0],
            "answer":k[2],
            "predicted":s,
            "linear_formula":k[1]
        }
        json_data.append(d)
    with open(str(name)+".json", 'w') as json_file:
        json.dump(json_data, json_file,indent=4)
    print("prediction json generated!!")

In [ ]:
data2string=generate_csv(model,train_data,train_loader)


In [ ]:
len(data2string)

In [ ]:
good=extract_sol(data2string)

In [ ]:
len(good)

In [ ]:
generate_sol(good,"train_data_model2",train_data)

In [ ]:
import csv
def gen_csv(l,name):
    csv_file = name+".csv"
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Loss"])  # Write header
        for loss in l:
            writer.writerow([loss])

In [ ]:
gen_csv(train_l,"train_loss_model_2")
gen_csv(val_l,"val_loss_model_2")
print("csv generated..")